### Scrape upcoming events

In [28]:
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup
import itertools
import string
from retrying import retry

# Scrape the events from the website

In [29]:
url = "http://www.tennisabstract.com/current"
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,/;q=0.8",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}
dft = pd.DataFrame()

In [30]:
def get_soup(url):
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Raise an exception for 4xx and 5xx status codes
    return BeautifulSoup(response.text, 'html.parser')

In [31]:
def get_current_event_data(html_content):
    event_data = []
    table = html_content.find('table')
    rows = table.find_all('tr')
    for row in rows[2:]:  # skipping the header rows
        columns = row.find_all('td')
        if len(columns) >= 2:
            event_name = columns[1].find('a').text.strip()
            event_date = columns[2].text.strip()
            href_link = columns[1].find('a')['href']
            event_data.append({'Event Name': event_name, 'Event Date': event_date, 'Href Link': href_link})
    return event_data


def write_events_to_csv(event_data, filename='../data/events.csv', overwrite=True):
    df = pd.DataFrame(event_data)
    print(df)


def main():
    url = "http://www.tennisabstract.com/current/"
    try:
        soup = get_soup(url)
        event_data = get_current_event_data(soup)
        dft = pd.DataFrame(event_data)
        print("TESt")
        print(dft)
        write_events_to_csv(event_data)
        print("Events data has been written to events.csv")
    except Exception as e:
        print("An error occurred:", e)



In [33]:
url = "http://www.tennisabstract.com/current/"
try:
    soup = get_soup(url)
    print(soup)
except Exception as e:
    print("An error occurred:", e)


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>Tennis Abstract: ATP Match Results, Splits, and Analysis</title>
<link href="http://www.minorleaguesplits.com/tennisabstract/blue/style.css" rel="stylesheet" type="text/css"/>
</head>
<body>
<table width="1000">
<tr>
<td>
<table width="500">
<tbody>
<tr>
<td align="left" class="headline" valign="top">
<h1>tennis<span style="color: blue">abstract</span>.com <em>beta</em></h1></td></tr>
<tr>
<td><input id="tags"/></td></tr>
<tr>
<td>
<p> <br/>
<strong>TennisAbstract.com</strong> is a new way to analyze ATP tennis players. Use the search box to find your favorite, 

or jump in with <a href="http://www.tennisabstract.com/cgi-bin/player.cgi?p=RadekStepanek">Radek Stepanek</a>. </p>
<p>The database currently consists of over 1600 players--everyone who has appeared in an ATP-level match since 1991.  

Match result

In [34]:
events = pd.read_csv('../data/events.csv')

EmptyDataError: No columns to parse from file

In [ ]:
# get only events that are happening now to 2 weeks ago to 1 week ahead

events['Event Date'] = pd.to_datetime(events['Event Date'])

events = events[(events['Event Date'] >= pd.Timestamp.now() - pd.DateOffset(weeks=2)) & (
            events['Event Date'] <= pd.Timestamp.now() + pd.DateOffset(weeks=1))]

events

Now we will check if there are any upcoming matches in the events

In [ ]:
base_url = 'http://www.tennisabstract.com/current/'
event_urls = events['Href Link'].tolist()


# def get_events_with_upcoming_matches(event_urls):
#     events_with_matches = []
#     for event_url in event_urls:
#             soup = get_soup(base_url + event_url)
#             matches = soup.find_all('span'  , {'id': 'upcoming'})
#             if len(matches) > 0:
#                 events_with_matches.append(event_url)
#     return events_with_matches


url = "https://www.tennisabstract.com/current/2024WTAParis125.html"
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"
}

# Make the request
response = requests.get(url, headers=headers)

# Define the pattern to match the upcomingSingles variable
upcoming_singles_matches = re.findall(r"var upcomingSingles = (.*?);", response.text)

print(upcoming_singles_matches)


In [ ]:
import re
import requests

# Define the URL and headers
url = "https://www.tennisabstract.com/current/2024WTAParis125.html"
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"
}

# Make the request
response = requests.get(url, headers=headers)

table = [["Event Name", "Player 1", "Player 2"]]  # Initialize the table

# Define the pattern to match the upcomingSingles variable
upcoming_singles_matches = re.findall(r"var upcomingSingles = (.*?);", response.text)

# Check if any matches are found
if upcoming_singles_matches:
    # Define the pattern to match each match in the list
    pattern_match = r'(.*?)\s*<br/>'
    # Process each match separately
    for match in upcoming_singles_matches:
        # Find all matches in the list
        matches = re.findall(pattern_match, match)
        # Create a table  
        # Process each match
        for m in matches:
            # Define the pattern to match player names within <a> tags
            pattern_player = r'<a href=".*?">(.*?)<\/a>'
            
            # Find all player names in the match
            player_names = re.findall(pattern_player, m)
            
            # Ensure there are even number of player names
            if len(player_names) % 2 != 0:
                player_names = player_names[:-1]  # Discard the last player if uneven
            
            # Extract player pairs
            player_pairs = [(player_names[i], player_names[i+1]) for i in range(0, len(player_names), 2)]
            
            # Add player pairs to the table
            for i, pair in enumerate(player_pairs):
                event_name = "WTAROME"
                table.append([event_name, pair[0], pair[1]])

# Print the table outside the loop
for row in table:
    print(row)


Now for all the events

In [ ]:
def get_upcoming_matches(event_urls):
    table = [["Event Name", "Player 1", "Player 2"]]  # Initialize the table
    for event_url in event_urls:
        url = base_url + event_url
        response = requests.get(url, headers=headers)
        upcoming_singles_matches = re.findall(r"var upcomingSingles = (.*?);", response.text)
        if upcoming_singles_matches:
            pattern_match = r'(.*?)\s*<br/>'
            for match in upcoming_singles_matches:
                matches = re.findall(pattern_match, match)
                for m in matches:
                    pattern_player = r'<a href=".*?">(.*?)<\/a>'
                    player_names = re.findall(pattern_player, m)
                    if len(player_names) % 2 != 0:
                        player_names = player_names[:-1]  # Discard the last player if uneven
                    player_pairs = [(player_names[i], player_names[i + 1]) for i in range(0, len(player_names), 2)]
                    for i, pair in enumerate(player_pairs):
                        event_name = event_url.split('.html')[0]
                        table.append([event_name, pair[0], pair[1]])
    return table


upcoming_matches = get_upcoming_matches(event_urls)

# convert to dataframe
upcoming_matches_df = pd.DataFrame(upcoming_matches[1:], columns=upcoming_matches[0])
upcoming_matches_df

In [ ]:
#upcoming_matches_df.to_csv('../data/upcoming_matches.csv', index=False)

In [ ]:
upcoming_matches_info = pd.read_csv('../data/upcoming_matches_info.csv', index_col=0)

In [ ]:
# remove the 'Augsburg CH' CH from the upcoming_matches_info.Tournament column
upcoming_matches_info['Tournament'] = upcoming_matches_info['Tournament'].str.replace(' CH', '')
upcoming_matches_info

In [ ]:
# join the two dataframes

In [ ]:
names = upcoming_matches_info['Results'].str.split('vs')
# remove everything between () and []
names = names.apply(lambda x: [re.sub(r'\[.*?\]', '', re.sub(r'\(.*?\)', '', name)).strip() for name in x])

In [ ]:
upcoming_matches_info['player1'] = names.apply(lambda x: x[0])
upcoming_matches_info['player2'] = names.apply(lambda x: x[1])

In [ ]:
players = pd.read_csv('../data/player_urls.csv')

In [ ]:
def find_matching_player_name(player_name):
    for player in players['name']:
        if all(part.lower() in player.lower() for part in player_name.split()):
            return player
    return None

In [ ]:
# Apply the function to create player_name column

upcoming_matches_info['player_name'] = upcoming_matches_info['player1'].apply(lambda x: find_matching_player_name(x) if pd.notnull(x) else None)
upcoming_matches_info = upcoming_matches_info.drop(columns=['player1', 'Results'])
upcoming_matches_info = upcoming_matches_info.rename(columns={'player_name': 'player1'})
upcoming_matches_info

In [ ]:
# rearrange the columns
upcoming_matches_info = upcoming_matches_info[['Tournament','Surface', 'Date', 'player1', 'player2', ]]
upcoming_matches_info

In [ ]:
for index, row in upcoming_matches_info.iterrows():
    for event in upcoming_matches_df['Event Name']:
        if row['Tournament'] in event:
            upcoming_matches_info.loc[index, 'Event'] = event
            
upcoming_matches_info

In [ ]:
upcoming_matches_info['Event'] = upcoming_matches_info['Event'].astype(str)

In [ ]:
def format_event(event):
    return re.sub(r'(?<=[0-9])(ATP|WTA)', r' \1', event)

In [ ]:
upcoming_matches_info['Event'] = upcoming_matches_info['Event'].apply(format_event)

upcoming_matches_info

In [ ]:
upcoming_matches_info = upcoming_matches_info.rename(columns={'Tournament': 'Location'})

In [ ]:
#upcoming_matches_info.to_csv('../data/upcoming_matches_info_clean.csv')

In [ ]:
# group by tournament
tournaments = upcoming_matches_info.copy()
tournaments = tournaments.groupby('Event').agg({'Location': 'first', 'Surface': 'first', 'Date': 'first'}).reset_index()
tournaments = tournaments.dropna()
tournaments

In [ ]:
#tournaments.to_csv('../data/tournaments.csv', index=False)